### Load the libiaries and the fire stations and property data  

In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import os
import read_data
import functions
import os
import re
import glob as glob

cwd = os.getcwd()
print(cwd)

states = ['NY']
#  CT, MA, NH, NJ, NY, PA



FS_path = "s3://pr-home-datascience/DSwarehouse/Datasources/FireStationLocation/FireStation/FireStations_0.csv"
# Use read_data.py to load the data
for state in states:
    property_path = "s3://pr-home-datascience/Users/test/IntermediateDrive/Quantarium/AD/good_address/rundt=202504/" + 'state='+states[0] + '/'
    df_FS = read_data.ReadData(FS_path, state = state, directory=False)

    df_property = read_data.ReadData(property_path, state=state, directory=True)
    print(f'There are {len(df_property.data)} data in the file.')

/home/sagemaker-user/DRIVING_DISTANCE
There are 6389671 data in the file.


### Complete the missing data
The big chunk request may have some missing data. Please check the files with "fail" keywords and complete them with the following function.

The default directory of chunks with missing data is './missing_data/' + state \
You can also use the Amazon S3 directory:\
missing_data_path = 's3://pr-home-datascience/Projects/AdHoc/InternProjects/2025/2025InternSummer Driving distance for Prospect Table/'+state+'/'


The default directory of chunks with complete data after reruning the request is './complete_data/' + state\
Or use the Amazon S3 directory:\
output_path = 's3://pr-home-datascience/Projects/AdHoc/InternProjects/2025/2025InternSummer Driving distance for Prospect Table/'+state+'/complete_data/'


Steps:\
1: Set the current state to process\
2: Define paths for missing data and output directory\
3: Retrieve all CSV files in the missing data folder\
4: Define chunksize per state (default to 300k for most states)\
5: Get total number of properties for this state\
6: Loop through each missing file to recompute using complete_osrm_results\
7: Rerun OSRM routing on the failed batch


In [5]:
# We are grouping the data by states
state = 'NY'

# the paths for missing data and for the results
missing_data_path = os.path.join('.', 'missing_data', state)

# output path to save the data
output_path = os.path.join('.', 'complete_data', state)

# obtain all the data file, format: "FS_batch_?_n"+chunksize +'failed.csv'
csv_files = glob.glob(os.path.join(missing_data_path, '*.csv'))
csv_files.sort()

# adjust chunksize based on the chunksize from property_FS.ipynb
chunksizes = {'MA': 100000, 'NJ': 200000, 'NY': 300000, 'PA': 300000, 'CT': 300000, 'NH': 300000}
# the length of total data
total = len(df_property.data)

# print(csv_files)
for file in csv_files:
# for file in csv_files[7:]:
    print('Currently handling the file ', file)
    chunksize = chunksizes[state]
    # get the batch index of the file, the index will be used to locate the property information.
    match = re.search(r'FS_batch_(\d+)_', file) 
    index = int(match.group(1))  

    # The chunk that needs to be filled
    start = index * chunksize
    end = min((index + 1) * chunksize, total)
    df_chunk = df_property.data.iloc[start:end]

    tmp = functions.complete_osrm_results(
        df_chunk,
        df_FS.data,
        file,
        output_path=output_path + f"/FS_batch_{index}_n{chunksize}.csv",
        batch_size=50,
        osrm_url = "http://router.project-osrm.org"
    )
    


['./missing_data/NY/FS_batch_0_n300000_failed.csv', './missing_data/NY/FS_batch_19_n300000_failed.csv', './missing_data/NY/FS_batch_1_n300000_failed.csv', './missing_data/NY/FS_batch_20_n300000_failed.csv', './missing_data/NY/FS_batch_21_n300000_failed.csv', './missing_data/NY/FS_batch_2_n300000_failed.csv']
Currently handling the file  ./missing_data/NY/FS_batch_19_n300000_failed.csv
[✓] Loading input data...
NaN in df_result: 0
Indices with NaN travel_time_min: []
Fill the NaN with 0.0.
[!] Missing OSRM results for 700 rows. Starting retry...


100%|██████████| 14/14 [02:26<00:00, 10.44s/it]


After merging the final data has length of 300000.
[✓] Done! Final results with 300000 rows saved to ./complete_data/NY/FS_batch_19_n300000.csv.


### Merge the data
Combine all the completed chunks, we need to verify the sequence of the property is the same as the original data, and merge them into one files.

Steps:\
1: Set the target state, load property data from S3 using read_data.py\
2: Define the output path for merged result\
3: Define a helper function to extract batch index from filenames\
4: Load all completed batch CSVs\
5: Get chunk size and total row count\
6: Optional - check for NaNs in travel_time_min in each batch file\
7: Merge all batches, verifying QPID alignment\
8: Check that the order of QPIDs matches the original dataset\
9: Concatenate all batches into a single DataFrame and add 'state' column


In [6]:
state = 'NY'
# Use read_data.py to load the data
property_path = "s3://pr-home-datascience/Users/test/IntermediateDrive/Quantarium/AD/good_address/rundt=202504/" + 'state='+state + '/'
df_property = read_data.ReadData(property_path, state=state, directory=True)

# output directory
output_path = 's3://pr-home-datascience/Projects/AdHoc/InternProjects/2025/2025InternSummer Driving distance for Prospect Table/results/'

# sort the results in the order of the batch index
def extract_batch_number(path):
    filename = os.path.basename(path)
    match = re.search(r'FS_batch_(\d+)_', filename)
    return int(match.group(1)) if match else -1

# The complete data directory
# The default is './complet_data/' + state + '/'
data_path = './complete_data/' + state
csv_files = glob.glob(os.path.join(data_path, '*.csv'))
csv_files.sort(key=extract_batch_number)

chunksizes = {'MA': 100000, 'NJ': 200000, 'NY': 300000, 'PA': 300000, 'CT': 300000, 'NH': 300000}
chunksize = chunksizes[state]

# the length of total data
total = len(df_property.data)

# to record the results after merging
all_batches = []

# check the NaN in each chunk
for file in csv_files:
    batch_index = extract_batch_number(file)
    df_batch = pd.read_csv(file)
    num_nan = df_batch["travel_time_min"].isna().sum()
    print(f"Number of NaN values in travel_time_min: {num_nan} in file", file)

# merging the data
for file in csv_files:
    # load chunks
    batch_index = extract_batch_number(file)
    df_batch = pd.read_csv(file)
    start = batch_index * chunksize
    end = min((batch_index + 1) * chunksize, total)

    # fill the NaN with 0.0 use if necessary
    # df_batch['travel_time_min'] = df_batch['travel_time_min'].fillna(0.0)
    # df_batch['travel_dist_mile'] = df_batch['travel_dist_mile'].fillna(0.0)

    ## The following is to verify the qpid are in the same order as the original property dataframe
    expected_qpids = df_property.data.iloc[start:end]['QPID'].reset_index(drop=True)
    batch_qpids = df_batch['qpid'].reset_index(drop=True)

    # The results shouls have the exact same qpid order with the original data
    # If failed, print out key messages
    if not batch_qpids.equals(expected_qpids):
        print(f"[ERROR] qpid mismatch in file: {file}")
        mismatch_mask = batch_qpids != expected_qpids
        mismatch_indices = mismatch_mask[mismatch_mask].index.tolist()
        print(f"Total mismatches: {len(mismatch_indices)}")

        # print the first 10 mismatches
        for i in mismatch_indices[:10]:
            print(f"  Index {i}: batch_qpid = {batch_qpids.iloc[i]}, expected_qpid = {expected_qpids.iloc[i]}")
    else:
        print(f"File {batch_index} qpid index match passed.")

    all_batches.append(df_batch)


# merge all data
df_merged = pd.concat(all_batches, ignore_index=True)
df_merged['state'] = state

# save to the file
df_merged.to_csv(output_path + state+'_FS_merged_all.csv', index=False)
print(f"Final merged file saved to: {output_path}")




Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_0_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_1_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_2_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_3_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_4_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_5_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_6_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_7_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_8_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/FS_batch_9_n300000.csv
Number of NaN values in travel_time_min: 0 in file ./complete_data/NY/

### Extract results examples
We will extract some examples from the estimations and the property information and verify them online. 

In [18]:
state = 'MA'

# Load the property information in state
property_path = "s3://pr-home-datascience/Users/test/IntermediateDrive/Quantarium/AD/good_address/rundt=202504/" + 'state='+state + '/'
df_property = read_data.ReadData(property_path, state=state, directory=True)
print(df_property.preview())

print('-----------seperation--------------------')
# Load the estimation from property to firestation in state
output_path = 's3://pr-home-datascience/Projects/AdHoc/InternProjects/2025/2025InternSummer Driving distance for Prospect Table/results/'
df_estimation = pd.read_csv(output_path+state+'_FS_merged_all.csv',  sep = ',', low_memory=False)
print(df_estimation.head())

print('-----------seperation--------------------')
# Load the fire station in state
FS_path = "s3://pr-home-datascience/DSwarehouse/Datasources/FireStationLocation/FireStation/FireStations_0.csv"
df_FS = read_data.ReadData(FS_path, state = state, directory=False)



       QPID State  PA_Latitude  PA_Longitude  Match_Code Location_Code  \
0  40136573    MA      42.4751      -70.9358           1           AS0   
1  42438550    MA      42.1538      -71.6540           1           AS0   
2  42438555    MA      42.1536      -71.6536           1           AS0   
3  64805738    MA      41.6612      -70.4364           1          AP02   
4  64805739    MA      41.7000      -70.2793           1           AS0   

  add_check geo_check  source   rundt state  
0      good         Y       2  202504    MA  
1      good         Y       2  202504    MA  
2      good         Y       2  202504    MA  
3      good         Y       2  202504    MA  
4      good         Y       2  202504    MA  
None
-----------seperation--------------------
       qpid  property_lat  property_lon  fire_station_id  station_lat  \
0  40136573       42.4751      -70.9358         10218765    42.479782   
1  42438550       42.1538      -71.6540         10229068    42.110870   
2  42438555  

In [20]:
print(np.argmax(df_estimation['travel_time_min']))
print(np.mean(df_estimation['travel_time_min']))
print(df_estimation.iloc[2090486])



# print(len(df_FS.data))

2090486
3.583302136084028
qpid                   67071259
property_lat          30.324418
property_lon         -81.664297
fire_station_id        10229509
station_lat           42.110551
station_lon           -73.35453
travel_time_min        1224.945
travel_dist_mile    1047.842102
state                        MA
Name: 2090486, dtype: object
